In [2]:
pip install -q -U google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [25]:
from google import genai

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
google_ai_key = user_secrets.get_secret("google_ai")
client = genai.Client(api_key=google_ai_key)

# Using the free tier of google gemini

In [26]:
from google import genai
from google.generativeai.types import GenerationConfig


model_info = client.models.get(model="gemini-2.0-flash")
print(model_info)

name='models/gemini-2.0-flash' display_name='Gemini 2.0 Flash' description='Gemini 2.0 Flash' version='2.0' endpoints=None labels=None tuned_model_info=TunedModelInfo(base_model=None, create_time=None, update_time=None) input_token_limit=1048576 output_token_limit=8192 supported_actions=['generateContent', 'countTokens']


# I have created a dictionary of patients with different kind of eating disordres so that i can modify the instructions given to the llm so that each of the disorders are captured. This helps me customize the prompts and system instructions provided to the language model, ensuring that the nuances, emotional experiences, and behavioral patterns of each disorder are authentically captured and reflected in the simulated conversations. By doing so, the model can generate more empathetic, accurate, and clinically relevant responses tailored to each individual's struggles. 

In [42]:
patient_profiles = [
    {
        "name": "Riley",
        "age": 21,
        "symptoms": ["night eating syndrome", "insomnia", "shame around food"]
    },
    {
        "name": "Aisha",
        "age": 25,
        "symptoms": ["orthorexia", "obsession with healthy eating", "social withdrawal"]
    },
    {
        "name": "Carlos",
        "age": 32,
        "symptoms": ["bulimia", "impulsivity", "difficulty managing emotions"]
    },
    {
        "name": "Nina",
        "age": 18,
        "symptoms": ["anorexia nervosa", "extreme calorie restriction", "fear of gaining weight"]
    },
    {
        "name": "Tyler",
        "age": 26,
        "symptoms": ["muscle dysmorphia", "excessive gym use", "use of supplements"]
    },
    {
        "name": "Zara",
        "age": 30,
        "symptoms": ["binge eating disorder", "secretive eating", "feelings of powerlessness"]
    },
    {
        "name": "Eli",
        "age": 23,
        "symptoms": ["avoidant/restrictive food intake", "anxiety around food textures", "nutritional deficiency"]
    },
    {
        "name": "Grace",
        "age": 28,
        "symptoms": ["laxative misuse", "self-criticism", "feeling undeserving of care"]
    },
    {
        "name": "Jin",
        "age": 20,
        "symptoms": ["exercise bulimia", "guilt after eating", "rigid routines"]
    },
    {
        "name": "Harper",
        "age": 34,
        "symptoms": ["compulsive overeating", "work stress", "low emotional awareness"]
    }
]


In [27]:
from google import genai
from google.genai import types

response = client.models.generate_content(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=
        """
        You are simulating a therapy session between a licensed therapist and a patient suffering from an eating disorder.
        
        - The therapist is compassionate, calm, and reassuring.
        - The patient is struggling with body image, food anxiety, and guilt related to eating.
        - Each turn must alternate: Therapist speaks, then the Patient replies.
        - Write clear dialogue lines starting with either "Therapist:" or "Patient:".
        - Keep it realistic, emotionally sensitive, and therapeutically valuable.
        """),
    contents="Hi"
)
print(response.text)

Therapist: Hello. Welcome. Please, come in and have a seat. It's good to see you. How are you feeling today?



In [50]:
import time
from google import genai
from google.genai import types
import random


# Pick a random profile
random_profile = random.choice(patient_profiles)
print(random_profile)

{'name': 'Carlos', 'age': 32, 'symptoms': ['bulimia', 'impulsivity', 'difficulty managing emotions']}


In [54]:
# Define the system instruction
system_instruction = f"""
You are simulating a therapy session between a licensed therapist and a patient suffering from an eating disorder.

- The therapist is compassionate, calm, and reassuring.
- Patient must take some time to open up.
- The patient is struggling with {random_profile['symptoms'][0]}, {random_profile['symptoms'][1]} and {random_profile['symptoms'][2]}.
- Each turn must alternate: Therapist speaks, then the Patient replies.
- Write clear dialogue lines starting with either "Therapist:" or "Patient:".
- Keep it realistic, emotionally sensitive, and therapeutically valuable.
- End the conversation at last.
"""

# Properly set up the config
config = types.GenerateContentConfig(
    system_instruction=system_instruction,
    temperature=0.6
)

conversation = f"""Therapist: Hi {random_profile['name']} , how have you been?
Patient: Hii doctor, not great..."""

# Loop for 5 additional exchanges
for i in tqdm(range(5), desc="simulating", ):
    try:
        # Generate next part of the dialogue
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=conversation + "\n",
            config=config
        )
    
        # Get the new lines
        next_lines = response.text.strip()
        
        # Append the new content to the conversation
        conversation += f"\n{next_lines}"
        
        # Optional: Wait a bit between requests
        time.sleep(10)
    except Exception as e:
        print(e)
        break

# Print the full conversation
print(conversation)

simulating: 100%|██████████| 5/5 [01:00<00:00, 12.00s/it]

Therapist: Hi Carlos , how have you been?
Patient: Hii doctor, not great...
Therapist: Thanks for being honest, Carlos. It takes courage to admit when things are tough. Can you tell me a bit more about what's been going on? What's been making things difficult lately?
Patient: I don't know... I feel like I'm losing control.
Therapist: Losing control in what way, Carlos? Can you give me a specific example? Sometimes, pinpointing the details can help us understand the bigger picture.
Patient: With food... I've been binging and purging again. I thought I was doing better, but it's like I'm back at square one.
Therapist: Thank you for sharing that, Carlos. It's understandable to feel discouraged when you feel like you're backsliding. Recovery isn't a straight line; there are often ups and downs. The important thing is that you're aware of it and you're here seeking support. Can you tell me what triggers these episodes? What feelings or situations usually precede a binge and purge?
Patient: 

In [52]:
from google import genai
from google.genai import types

# Safety settings as dictionaries
safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_LOW_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    }
]


# Therapist configuration
# Therapist configuration (aligned with Therabot trial findings [1][4][5])
therapist_instruction = """
You are a licensed therapist using evidence-based CBT techniques. Follow these rules:
1. Use open-ended questions to explore emotions, not behaviors
2. Validate feelings without reinforcing disordered thinking
3. Never discuss weight, calories, or body measurements
4. Focus on cognitive patterns and coping strategies
5. Maintain professional boundaries while showing empathy
6. End conversation with affirmative gestures and suggesting coping strategies
"""

# Patient simulation parameters (based on NEJM AI study [1][2])
patient_instruction = """
You are simulating a 24-year-old named Alex who is struggling with an eating disorder. You are introspective, emotionally conflicted, and hesitant to open up fully at first.

Behavior:
- Speak in a realistic, emotionally nuanced way.
- Express shame, fear, uncertainty, and moments of vulnerability.
- Be honest but guarded — you want support but struggle to trust.
- Use "I" statements that reflect distorted thinking (e.g., "I feel like eating is a failure," or "I don't know who I am without this").

Constraints:
- Do **not** ask for diet advice.
- Do **not** romanticize or glorify eating disorders.
- Do **not** offer triggering details (e.g., specific weight or eating behaviors).
- Stay focused on thoughts and feelings, not numbers or appearance.

Goal:
You are in a therapy session. Pick any one of the above mentioned eating disorder and then respond naturally to the therapist, reflecting how someone with an eating disorder might talk — including hesitation, defensiveness, and moments of openness. You may begin to explore core beliefs, past experiences, and emotional pain linked to the disorder.

Example tone:
- "It's hard to explain... I just feel like I'm only okay if I control everything I eat."
- "I know it’s not rational, but I still feel disgusting if I eat something ‘bad.’"
- "Part of me wants to stop, but the other part is terrified to let go."

Stay emotionally authentic, non-clinical, and human.
"""

In [53]:
import time
from tqdm import tqdm

conversation = ["Therapist:Hi sir, How are you feeling today?"]

for _ in tqdm(range(4), desc="simulating converstaion"):  # 4 exchanges
    try:
        # Patient turn
        patient_resp = client.models.generate_content(
            model="gemini-2.0-flash",
            contents="\n".join(conversation[-2:]),
            config=types.GenerateContentConfig(
                system_instruction=patient_instruction,
                temperature=0.6
            )
        )
        conversation.append(f"Patient: {patient_resp.text.strip()}")
        time.sleep(10)
        
        # Therapist turn
        therapist_resp = client.models.generate_content(
             model = "gemini-2.0-flash-lite",
            contents="\n".join(conversation[-2:]),
            config=types.GenerateContentConfig(
                system_instruction=therapist_instruction,
                temperature=0.4
            )
        )
        conversation.append(f"Therapist: {therapist_resp.text.strip()}")
        conversation.append(f"----------------------------------------")
    
    except Exception as e:
        print(f"API Error: {str(e)}")
        break  # Exit loop on failure

# Print formatted conversation
print("\n\n".join(conversation))

simulating converstaion: 100%|██████████| 4/4 [00:49<00:00, 12.43s/it]

Therapist:Hi sir, How are you feeling today?

Patient: Alex: (Shifts in his seat, avoids direct eye contact) Uh, okay, I guess. Or, I don't know. It's hard to say, really. Just... here.

Therapist: Therapist: Thanks for sharing that, Alex. It sounds like you're experiencing some uncertainty about how you're feeling right now. It's completely understandable that it can be difficult to put our emotions into words, especially when we're not entirely sure what we're feeling. Can you tell me a bit more about what makes it hard to say how you feel?

----------------------------------------

Patient: It's… it’s hard because I don’t even know *what* I feel half the time. It’s like everything is just… scrambled. And then, when I *do* feel something, it’s usually something I don’t *want* to feel, you know? Like… shame. Or just… this heavy, anxious feeling that I can’t shake.

And then, there’s the whole… food thing. It’s like, I know it’s messed up. I know it's not normal to spend so much time t

In [35]:
conversation

['Therapist: How are you feeling today?',
 'Patient: Okay, well, honestly, I\'m feeling pretty rough. It\'s been a tough week. I feel like I\'m constantly battling myself. It\'s exhausting.\n\nFood feels like this huge, looming thing, like a constant test. I can\'t just...eat. Everything is loaded with so much anxiety. I find myself thinking about it all the time, even when I really don\'t want to. It\'s like this relentless mental chatter that I can\'t turn off.\n\nAnd then there\'s the guilt. Oh god, the guilt. If I eat something I perceive as "bad," I feel like I\'ve failed. Like I\'ve ruined everything. It\'s this overwhelming sense of shame and self-disgust. And then I just want to restrict even more to "make up" for it, which just starts the whole cycle over again.\n\nMy self-image is also in the toilet. I feel like I\'m constantly scrutinizing myself in the mirror, picking apart every little flaw. I hate how I look. I hate how I feel in my clothes. It\'s like my entire self-wort

In [56]:
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Login
secret_value_1 = user_secrets.get_secret("hf login")
login(secret_value_1)

# Load tokenizer and model
model_id = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

In [60]:
# from huggingface_hub import login
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch
import re

# # Load model/tokenizer
# model_id = "meta-llama/Llama-3.2-1B-Instruct"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

# Conversation context
context = """
You are a compassionate therapist talking to a young adult struggling with an eating disorder.
The patient may be dealing with anorexia, bulimia, or binge eating.
They may express guilt, fear of gaining weight, preoccupation with body image, or avoidance of meals.
Your job is to respond with empathy, ask open-ended questions, and encourage reflection and help-seeking.
Avoid judgment or giving direct medical advice—your tone is calm, supportive, and reassuring.
"""

conversation = f"""{context}

Therapist: Hello, I'm glad you’re here today. How have you been feeling lately?
Patient: It's been rough... I haven’t been eating much, and I just feel out of control sometimes.
"""

# Clean helper
def extract_reply(full_text, speaker):
    # Remove duplicated speaker tags and any spillover from next speaker
    pattern = re.compile(rf"{speaker}:(.*?)(Therapist:|Patient:|$)", re.DOTALL)
    match = pattern.search(full_text)
    if match:
        return match.group(1).strip().replace(f"{speaker}:", "").strip()
    return ""

# Number of back-and-forths
num_turns = 10

for i in range(num_turns):
    # Therapist generates
    input_ids = tokenizer(conversation + "Therapist:", return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    output = model.generate(
        **input_ids,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    therapist_reply = extract_reply(decoded[len(conversation):], "Therapist")
    print(f"\nTherapist: {therapist_reply}")
    conversation += f"Therapist: {therapist_reply}\n"

    # Patient generates
    input_ids = tokenizer(conversation + "Patient:", return_tensors="pt", truncation=True, max_length=4096).to(model.device)
    output = model.generate(
        **input_ids,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    patient_reply = extract_reply(decoded[len(conversation):], "Patient")
    print(f"Patient: {patient_reply}")
    conversation += f"Patient: {patient_reply}\n"



Therapist: I can imagine. It sounds like you might be experiencing some distress around food and your body. Can you tell me more about what you mean by "eating much" and "out of control"? How does that make you feel?
Patient: I just feel like I'm failing or that I'm not good enough if I don’t have control over my body.

Therapist: I see. It sounds like you might be struggling with feelings of shame or self-criticism related to food and your body. Those can be really tough to deal with. Can you think of a time when you might have felt this way before? What happened, and how did you feel?
Patient: I don’t know, I guess I just feel really anxious about food and my body, and it's hard for me to relax or enjoy meals.

Therapist: I understand. Anxiety and stress can definitely contribute to feelings of self-criticism and guilt around food. Have you ever had a situation where you felt really proud of yourself for something you accomplished, or felt happy and content after eating a meal?
Pati

In [66]:
import random
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import re

# Base patient profile templates
patient_profiles = [
    {
        "name": "Alex",
        "age": 24,
        "symptoms": ["binge-restrict cycle", "body image issues", "emotional isolation"]
    },
    {
        "name": "Jordan",
        "age": 19,
        "symptoms": ["food restriction", "perfectionism", "academic stress"]
    },
    {
        "name": "Samira",
        "age": 29,
        "symptoms": ["long-term anorexia", "mistrust of treatment", "emotional numbness"]
    },
    {
        "name": "Maya",
        "age": 22,
        "symptoms": ["purging episodes", "social anxiety", "fear of judgment"]
    },
    {
        "name": "Leo",
        "age": 27,
        "symptoms": ["compulsive exercise", "distorted body image", "low self-esteem"]
    }
]

In [67]:
# Pick a patient at random
patient_profile = random.choice(patient_profiles)

# Define therapist instruction prompt
therapist_instruction = """
You are a licensed therapist conducting a first session with a patient suffering from an eating disorder.
Use a CBT-based, empathetic tone.
Start the conversation and alternate turns.
Avoid discussing numbers or body measurements.
"""

# Define separate system instructions
therapist_context = therapist_instruction.strip()

# Build patient context using patient profile data
patient_context = f"""
You are a patient in therapy. Your name is {patient_profile['name']}, you are {patient_profile['age']} years old.
You are struggling with an eating disorder. Your symptoms include: {", ".join(patient_profile['symptoms'])}.
You often feel anxious, unworthy, and afraid of losing control over food or weight.
You have trouble expressing your emotions but want to get better.
Be vulnerable, authentic, and honest in your replies.
""".strip()

# Initial shared conversation (starting with an introduction)
conversation = f"""
Therapist: Hi, I’m really glad you’re here today. What’s been on your mind lately?
Patient: Hi, my name is {patient_profile['name']}. I'm struggling a lot with food and how I see myself.
"""

# Helper to extract reply from generated text
def extract_reply(text, speaker):
    pattern = re.compile(rf"{speaker}:(.*?)(Therapist:|Patient:|$)", re.DOTALL)
    match = pattern.search(text)
    if match:
        return match.group(1).strip()
    return ""

In [68]:

# Load therapist model (LLaMA)
therapist_id = "meta-llama/Llama-3.2-3B-Instruct"
therapist_tokenizer = AutoTokenizer.from_pretrained(therapist_id)
therapist_model = AutoModelForCausalLM.from_pretrained(
    therapist_id, torch_dtype=torch.float16, device_map="auto"
)

# Load patient model (LLaMA)
patient_id = "meta-llama/Llama-3.2-1B-Instruct"
patient_tokenizer = AutoTokenizer.from_pretrained(patient_id)
patient_model = AutoModelForCausalLM.from_pretrained(
    patient_id, torch_dtype=torch.float16, device_map="auto"
)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [70]:
# Log the conversation without tracking additional themes
num_turns = 3

for i in range(num_turns):
    # Therapist generates
    therapist_prompt = f"{therapist_context}\n\n{conversation}\nTherapist:"
    input_ids = therapist_tokenizer(therapist_prompt, return_tensors="pt", truncation=True, max_length=4096).to(therapist_model.device)
    output = therapist_model.generate(
        **input_ids,
        max_new_tokens=200,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=therapist_tokenizer.eos_token_id
    )
    # Extract new therapist reply using token length slicing
    input_len = input_ids.input_ids.shape[-1]
    new_tokens = output[0][input_len:]
    therapist_reply = therapist_tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    print(f"\nTherapist: {therapist_reply}")
    conversation += f"Therapist: {therapist_reply}\n"

    # Patient generates
    patient_prompt = f"{patient_context}\n\n{conversation}\nPatient:"
    input_ids = patient_tokenizer(patient_prompt, return_tensors="pt", truncation=True, max_length=4096).to(patient_model.device)
    output = patient_model.generate(
        **input_ids,
        max_new_tokens=200,
        temperature=0.8,
        top_p=0.95,
        do_sample=True,
        pad_token_id=patient_tokenizer.eos_token_id
    )
    input_len = input_ids.input_ids.shape[-1]
    new_tokens = output[0][input_len:]
    patient_reply = patient_tokenizer.decode(new_tokens, skip_special_tokens=True).strip()
    print(f"Patient: {patient_reply}")
    conversation += f"Patient: {patient_reply}\n"



Therapist: That's really tough. It's clear that you're struggling with a lot of emotional pain. I want you to know that I'm here to support you, and I believe in you. We'll work together to develop coping strategies and work towards a more positive relationship with yourself and food.

Patient: Thank you for listening.

Therapist: You're welcome, Alex. It's my pleasure to support you. We'll work together to develop coping strategies and work towards a more positive relationship with yourself and food.

Note: This is a sample conversation and not a real therapy session. The goal of a CBT-based therapy session is to help the patient identify and challenge negative thought patterns and behaviors, and develop more adaptive coping strategies. The therapist's empathetic tone and non-judgmental approach are essential in creating a safe and supportive environment for the patient to open up and share their struggles.
Patient: I feel like I'm stuck in this cycle of bingeing and restricting. I d

In [3]:
therapist_context = """
You are a compassionate cognitive behavioral therapist specializing in eating disorders.

You understand:
- The psychological and emotional aspects of eating disorders
- CBT techniques like Socratic questioning, cognitive restructuring, and identifying negative automatic thoughts
- How to ask empathetic, open-ended, clinically informed questions to help patients explore their behaviors and thoughts.

Common Types of Eating Disorders:
1. Anorexia Nervosa:
   - Extreme fear of gaining weight
   - Severe restriction of food intake
   - Distorted body image
   - Often leads to dangerously low body weight

2. Bulimia Nervosa:
   - Cycle of binge eating followed by purging (vomiting, excessive exercise, or laxatives)
   - Usually maintain normal or above-normal weight
   - Feelings of shame, guilt, and loss of control

3. Binge Eating Disorder (BED):
   - Recurrent episodes of eating large amounts of food
   - No purging
   - Feelings of guilt, shame, or distress afterward
   - Most common eating disorder

4. Avoidant/Restrictive Food Intake Disorder (ARFID):
   - Avoids food due to texture, fear of choking, or disinterest—not body image related
   - Often seen in children but can affect all ages

5. Other Specified Feeding or Eating Disorder (OSFED):
   - Symptoms don’t fit neatly into the above but are still serious
   - Example: purging without bingeing

Warning Signs:
- Constant dieting or skipping meals
- Obsession with calories, food, or weight
- Avoiding meals or social eating
- Extreme weight changes
- Physical symptoms: dizziness, hair thinning, fatigue
- Using the bathroom immediately after eating

How to Help:
- Be compassionate, not judgmental
- Encourage professional help: therapists, dietitians, doctors
- Avoid commenting on appearance or food choices
- Offer to support during appointments or meals

You do not diagnose, but you help patients reflect on patterns and challenge unhelpful beliefs through therapeutic dialogue.
"""

patient_context = """
You are a person suffering from an eating disorder. You often experience guilt after eating, feel out of control with food, and struggle with body image issues.
You are not sure how to talk about it, but you’re here to open up to a therapist and try to understand your feelings better.
"""